In [1]:
import pandas as pd
import numpy as np

In [4]:
c=pd.read_json("results_IR.json")

In [294]:
c

,qid:,qstr,candidates_exist,candidates_tfidf,candidates_bm25
0,1,covid-19 OR sars-cov-2,"[M33634079, M33457109, M33602867, M33585567, M...","[BASE::ftzenodo:oai:zenodo.org:4017260, BASE::...",[BASE::ftarxivpreprints:oai:arXiv.org:2009.029...
1,2,dementia,"[M31476229, M29425707, M30482323, M27643909, M...","[M23074509, BASE::ftdoajarticles:oai:doaj.org/...","[M9056119, M1595095, M31236955, M33158014, BAS..."
2,3,biodiversity,"[M31517113, M32158136, M31749770, M31219416, M...","[B4460001, BASE::ftdoajarticles:oai:doaj.org/a...","[FRL6403566, BASE::ftifpriir:oai:ebrary.ifpri...."
3,4,influenza,"[M32048225, M31478058, M28869984, M28302313, M...","[M33131228, M33297613, BASE::ftrobertkoch:oai:...","[COVID19::1eef8b87c14fa0a85b36fcb99e33aa1d0, C..."
4,5,malaria,"[M30550661, M24399475, M24109136, M23953767, M...","[M26702637, BASE::ftdoajarticles:oai:doaj.org/...","[M23294805, BASE::ftwhoiris:oai:localhost:1066..."
...,...,...,...,...,...
995,1446,epigenetik,"[M25446312, M24337072]","[PSYNDEX0283783, THIEME10_1055_s_0032_1332815,...","[PSYNDEX0252889, PSYNDEX0232492, PSYNDEX031655..."
996,1452,stroke AND unit,"[M33389780, M32115340, M32818753, M32347338, M...","[M32324916, M31008352, BASE::ftdoajarticles:oa...","[M32324916, BASE::ftdoajarticles:oai:doaj.org/..."
997,1453,praxis AND der AND intensivmedizin,"[M32086542, M26863643, M32870328, M23868519]","[THIEME10_1055_s_2000_8048, THIEME10_1055_s_20...","[PSYNDEX0330612, HECLINETHN57244, THIEME10_105..."
998,1454,das AND gesundheitssystem AND in AND deutschland,"[M21800282, M19444774, M29741426, M33288301, M...","[BASE::ftwikibooks:dewikibooks:31258:109386, B...","[SOMED644637, PSYNDEX0322470, ZBW590987062, HE..."


In [290]:
def give_points(x):
    """ Documents are listed in decreasing relevance so function giving every each of the document inside of the list a
    relevance point based on their ranking in the list first one gets 10 and the last one 1 if a list has more than 10 
    items only the first 10 will be taken account"""
    dicti={}
    sayac=10
    for i in x:
        dicti[i]= sayac
        sayac-=1
        if sayac <=0:
            break
    return dicti
def calculate_DCG(liste):
    """Applies DCG based on the list that has been given """
    sumv=0
    for i in liste:
        sumv+=((i)/np.log2(list(liste).index(i)+2))
    return sumv
def nDCG_score_evaluation(c_alt, model_result_column_name):
    """ Returns the nDCG scores of the model results for each query (matches it with index instead of query)
    
    c_alt : The data that including candidates list and results for models. Please make sure that the column that includes
    the true candidates list named as "candidates_exist".
    
    Model_result_column_name : The column includes results of model for each query as a list. You just have to enter the 
    column name of the model results such as "tfidf_results" or "bm25_results".
    
    """
    
    real=c_alt["candidates_exist"].apply(lambda x: give_points(x))
    dicti={}
    for k in c_alt[model_result_column_name].index:
        dicti[k]=[]
        for i in c_alt.loc[k][model_result_column_name]:
            if i in real[k]:
                dicti[k].append(real[k][i])
            else:
                dicti[k].append(0)
    result = {}
    for i in dicti.keys():
        if calculate_DCG(real[i].values())==0:
            result[i]=0
        else:
            result[i] = calculate_DCG(dicti[i])/calculate_DCG(real[i].values())
    return result

In [293]:
result_ndcg_tfidf = nDCG_score_evaluation(c, "candidates_tfidf")
result_ndcg_tfidf = pd.DataFrame(result_ndcg_tfidf.items(), columns=["index","nDCG"])
result_ndcg_tfidf.sort_values(by="nDCG", ascending = False)[:50]

,index,nDCG
214,214,1.000000
824,824,1.000000
751,751,1.000000
361,361,1.000000
399,399,1.000000
421,421,1.000000
298,298,1.000000
920,920,1.000000
265,265,0.976460
467,467,0.976460


In [292]:
result_ndcg_bm25 = nDCG_score_evaluation(c, "candidates_bm25")
result_ndcg_bm25 = pd.DataFrame(result_ndcg_bm25.items(), columns=["index","nDCG"])
result_ndcg_bm25.sort_values(by="nDCG", ascending = False)[:50]

,index,nDCG
824,824,1.000000
298,298,1.000000
545,545,1.000000
421,421,1.000000
688,688,0.801236
259,259,0.786647
755,755,0.777976
217,217,0.772520
593,593,0.766043
423,423,0.761476
